In [12]:

# let's import libraries
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
# this is the file you should've gotten from your repo
from utils import extract_dates, get_val_scores
from category_encoders import OrdinalEncoder, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, train_test_split


In [34]:
df = pd.read_csv('../data/ks2.csv',encoding='utf-8',parse_dates=['deadline','launched'])

In [14]:
df.head()

,ID,name,category,main_category,currency,deadline,launched,state,country,goal
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,2015-08-11 12:12:28,failed,GB,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,2017-09-02 04:43:57,failed,US,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,2013-01-12 00:20:50,failed,US,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,2012-03-17 03:24:11,failed,US,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,2015-07-04 08:35:03,failed,US,19500.00


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 370454 entries, 0 to 370453
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   ID             370454 non-null  int64         
 1   name           370451 non-null  object        
 2   category       370454 non-null  object        
 3   main_category  370454 non-null  object        
 4   currency       370454 non-null  object        
 5   deadline       370454 non-null  datetime64[ns]
 6   launched       370454 non-null  datetime64[ns]
 7   state          370454 non-null  object        
 8   country        370454 non-null  object        
 9   goal           370454 non-null  float64       
dtypes: datetime64[ns](2), float64(1), int64(1), object(6)
memory usage: 28.3+ MB


In [11]:
df.state.value_counts()

failed        236498
successful    133956
Name: state, dtype: int64

***
With this data set, we plan to predict whether it was failed or successful using a Gradient Boosting approach.

In [16]:
??extract_dates #Two "?" shows the source code of the function.

In [17]:
extract_dates(df,date_parts=['month','day','year'])

,ID,name,category,main_category,currency,deadline,launched,state,country,goal,deadline_month,deadline_day,deadline_year,launched_month,launched_day,launched_year,deadline_history_days,launched_history_days
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,2015-08-11 12:12:28,failed,GB,1533.95,10,9,2015,8,11,2015,2350,16658
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,2017-09-02 04:43:57,failed,US,30000.00,11,1,2017,9,2,2017,3104,17411
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,2013-01-12 00:20:50,failed,US,45000.00,2,26,2013,1,12,2013,1395,15716
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,2012-03-17 03:24:11,failed,US,5000.00,4,16,2012,3,17,2012,1079,15416
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,2015-07-04 08:35:03,failed,US,19500.00,8,29,2015,7,4,2015,2309,16620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370449,999976400,ChknTruk Nationwide Charity Drive 2014 (Canceled),Documentary,Film & Video,USD,2014-10-17,2014-09-17 02:35:30,failed,US,50000.00,10,17,2014,9,17,2014,1993,16330
370450,999977640,The Tribe,Narrative Film,Film & Video,USD,2011-07-19,2011-06-22 03:35:14,failed,US,1500.00,7,19,2011,6,22,2011,807,15147
370451,999986353,Walls of Remedy- New lesbian Romantic Comedy f...,Narrative Film,Film & Video,USD,2010-08-16,2010-07-01 19:40:30,failed,US,15000.00,8,16,2010,7,1,2010,470,14791
370452,999987933,BioDefense Education Kit,Technology,Technology,USD,2016-02-13,2016-01-13 18:13:53,failed,US,15000.00,2,13,2016,1,13,2016,2477,16813


In [23]:
df = extract_dates(df,cols=['deadline'], drop_cols=True)
# drop_cols = True will remove the columns mentioned

/Users/PRSmb/OneDrive/General-Assembly/my-1019-repo/ClassMaterial/Unit3/Class15/utils.py:28: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df[col_name] = getattr(df[col].dt, part)


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 370454 entries, 0 to 370453
Data columns (total 28 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   ID                         370454 non-null  int64         
 1   name                       370451 non-null  object        
 2   category                   370454 non-null  object        
 3   main_category              370454 non-null  object        
 4   currency                   370454 non-null  object        
 5   launched                   370454 non-null  datetime64[ns]
 6   state                      370454 non-null  object        
 7   country                    370454 non-null  object        
 8   goal                       370454 non-null  float64       
 9   deadline_dayofweek         370454 non-null  int64         
 10  deadline_dayofyear         370454 non-null  int64         
 11  deadline_days_in_month     370454 non-null  int64   

In [26]:
df['deadline_history_days'].min()

0

In [28]:
df['deadline_history_days'].max() 
#Effectively the length in days of our data sets.
# Allows us to numerically track the passage of time.

3219

In [29]:
df.size

10372712

In [31]:
df.nunique().sum() # Potential number of cut points.

1161303

In [32]:
# A tree 6 levels deep has
2**6 
# leaves

64

In [33]:
# So any given tree will only use a small portion of the columns
2**6 / df.nunique().sum() 

5.51105094880492e-05

In [ ]:
# So our models tend to end up insensitive to the number of columns

In [35]:
df = extract_dates(df)

In [36]:
# Initialise our XGB model
mod1 = xgb.XGBClassifier()

In [37]:
mod1.get_params()

{'objective': 'binary:logistic',
 'use_label_encoder': True,
 'base_score': None,
 'booster': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'gamma': None,
 'gpu_id': None,
 'importance_type': 'gain',
 'interaction_constraints': None,
 'learning_rate': None,
 'max_delta_step': None,
 'max_depth': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'n_estimators': 100,
 'n_jobs': None,
 'num_parallel_tree': None,
 'random_state': None,
 'reg_alpha': None,
 'reg_lambda': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

In [39]:
pipe = make_pipeline(OrdinalEncoder(),mod1)

In [40]:
X = df.drop(['deadline','launched','state'],axis=1)
y = df['state']

In [43]:
??get_val_scores

In [44]:
scores = get_val_scores(
    pipe, 
    X, 
    y, 
    return_test_score=True,
    random_state=1985, 
    stratify=True, 
    use_kfold=False
    )

/usr/local/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:43:20] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/usr/local/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


[12:43:53] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [45]:
scores

{'validation_score': 0.7277343815902687, 'test_score': 0.7005304287970199}

In [46]:
feats = pd.DataFrame({ 'Importance': pipe.steps[1][1].feature_importances_, 'Column': X.columns})
feats.sort_values(by='Importance', ascending=False)

,Importance,Column
6,0.123426,goal
3,0.087642,main_category
2,0.082025,category
43,0.053122,deadline_history_days
41,0.050469,launched_month
30,0.041295,launched_is_month_start
39,0.041036,launched_hour
44,0.036439,launched_history_days
4,0.035812,currency
5,0.034382,country


In [47]:
# How long is the campaign
df['duration'] = (df['deadline'] - df['launched']).dt.days

In [48]:
# Get the average for each category
category_avgs = df.groupby('category')[['goal']].mean().reset_index().rename({'goal':'category_goal_avg'},axis=1)

In [49]:
category_avgs

,category,category_goal_avg
0,3D Printing,47096.558771
1,Academic,55365.812915
2,Accessories,14737.425295
3,Action,320007.524411
4,Animals,10788.377328
...,...,...
154,Woodworking,11655.184791
155,Workshops,9739.618038
156,World Music,28711.839405
157,Young Adult,40515.781588


In [50]:
# Merge back into our data set
df = df.merge(category_avgs, on='category')

In [51]:
# and look at how a campaign ended up looking relative to the category average
df['cat_goal_pct'] = df['goal'] / df['category_goal_avg']

In [52]:
df[['goal','category_goal_avg','cat_goal_pct']].head()

,goal,category_goal_avg,cat_goal_pct
0,1533.95,5213.996468,0.294199
1,6060.97,5213.996468,1.162442
2,2000.00,5213.996468,0.383583
3,10000.00,5213.996468,1.917915
4,757.52,5213.996468,0.145286


In [53]:
# Run model again
X = df.drop(['deadline','launched','state'],axis=1)
y = df['state']

scores = get_val_scores(
    pipe, 
    X, 
    y, 
    return_test_score=False,
    random_state=1985, 
    stratify=True, 
    use_kfold=False
    )

/usr/local/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/usr/local/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:59:32] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [54]:
scores
# Wrap up for Class 15.

{'validation_score': 0.7296408145361294}